In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from pathlib import Path
from tqdm import tqdm
N = 12
df_trans = pd.read_csv("C:/Users/Michael/Github/transactions_train.csv",dtype={'article_id': str})
df_trans['t_dat'] = pd.to_datetime(df_trans['t_dat'])

In [2]:
# Step1
df = df_trans[['t_dat', 'customer_id', 'article_id']].copy()

In [3]:
last_ts = df['t_dat'].max()

In [4]:
df['ldbw'] = df['t_dat'].apply(lambda d: last_ts - (last_ts - d).floor('7D'))


KeyboardInterrupt



In [ ]:
df_cust = pd.read_csv("C:/Users/Michael/Github/customers.csv")

In [ ]:
df.head()

In [ ]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count()

In [ ]:
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

In [ ]:
df = df.join(weekly_sales, on=['ldbw', 'article_id'])

In [ ]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')

In [ ]:
last_day = last_ts.strftime('%Y-%m-%d')

In [ ]:
df = df.join(
    weekly_sales.loc[weekly_sales['ldbw']==last_day, ['count']],
    on='article_id', rsuffix="_targ")

In [ ]:
df['count_targ'].fillna(0, inplace=True)

In [ ]:
del weekly_sales

In [ ]:
df['quotient'] = df['count_targ'] / df['count']

In [ ]:
# purchase_dict = {}

In [ ]:
# for i in tqdm(df.index):
#     cust_id = df.at[i, 'customer_id']
#     art_id = df.at[i, 'article_id']
#     t_dat = df.at[i, 't_dat']

#     if cust_id not in purchase_dict:
#         purchase_dict[cust_id] = {}

#     if art_id not in purchase_dict[cust_id]:
#         purchase_dict[cust_id][art_id] = 0
    
#     x = max(1, (last_ts - t_dat).days)

#     a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
#     y = a / np.sqrt(x) + b * np.exp(-c*x) - d

#     value = df.at[i, 'quotient'] * max(0, y)
#     purchase_dict[cust_id][art_id] += value


In [ ]:
# target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()

In [ ]:
# general_pred = target_sales.nlargest(N).index.tolist()

In [ ]:
general_pred = ['706016001 706016002 610776002 610776002 372860001 759871002 610776001 673677002 399223001 156231001 372860002 448509014']

In [19]:
# Step2 & Step3
pairs = np.load('pairs.npy',allow_pickle=True).item()
sub = pd.read_csv("C:/Users/Michael/Github/sample_submission.csv")

In [20]:


pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in purchase_dict:
        series = pd.Series(purchase_dict[cust_id])
        series = series[series > 150]
        l = series.nlargest(N).index.tolist()
        tmp_l = l.copy()
        for elm in tmp_l:
            if len(l) < N and int(elm) in pairs.keys():
                itm = pairs[int(elm)]
                l.append('0' + str(itm))
        if len(l) < N:
            l = l + general_pred[:(N-len(l))]
    else:
        l = general_pred
    pred_list.append(' '.join(l))

sub['prediction'] = pred_list
sub.to_csv(f'submission_other_sub.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████| 1371980/1371980 [10:55<00:00, 2092.82it/s]


In [21]:
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0568601006 0568597006 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0706016001 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992033 0720125039 0713997002 0557599022 07...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0818754002 0762846031 0759871030 0624486088 08...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0884081001 0762846027 0794819001 0689365050 06...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0882810001 0714790020 0464297007 0448509014 04...
